In [1]:
import numpy as np
import numpy.ma as ma
import time
import os
import netCDF4 as nc
import matplotlib.pyplot as plt
%matplotlib notebook
from shutil import copyfile
from bootstrap import *

In [2]:
#years of interest - start and end date, with index starting at 1 out of habit.
myfilename = '/Users/Siwen/Desktop/Ferret/bin/P_asia_year.nc'

rainfile = nc.Dataset(myfilename, 'r')

P_asia = np.swapaxes(rainfile['P_asia_year'][:], 0, 1)
P_asia_sum = np.swapaxes(rainfile['P_asia_year_sum'][:], 0, 1)

p1_yr_b = [1,30]
p1_yr_e = [29,43]
p2_yr_b = [30,44]
p2_yr_e = [57,57]

P_asia_diff = []
P_asia_sum_diff = []

#cycle through all time periods of interest and find changes in each type of rainfall b/w periods
for p1_b, p1_e, p2_b, p2_e, i in zip(p1_yr_b, p1_yr_e, p2_yr_b, p2_yr_e, np.arange(len(p1_yr_b))): 
    
    P_asia_diff.append(np.mean(P_asia[:,:,p2_b-1:p2_e], axis=2) - \
                         np.mean(P_asia[:,:,p1_b-1:p1_e], axis=2))
    
    P_asia_sum_diff.append(np.mean(P_asia_sum[:,:,p2_b-1:p2_e], axis=2) - \
                         np.mean(P_asia_sum[:,:,p1_b-1:p1_e], axis=2))

In [18]:
#BOOTSTRAPPING

#similar to china_rain_diff, do lots of bootstrapping at each spatial point.

#daysmth is defined above
niter = 2000 #how many iterations each time?
filepath = "asia_rain_diff_pval_"
backuppath = "/Users/Siwen/Desktop/Ferret/bin/"

#netCDF output file formatting formatting
#yearsout = str(p1_b+50) + str(p1_e+50) + "_" + str(p2_b+50) + str(p2_e+50) + "_"
fullname = filepath + str(niter) + "iter.nc"
backupname = backuppath + fullname

##the following is only to make sure that we don't exceed the bounds of the number of rows of actual data.
if os.path.isfile(fullname):
    os.remove(fullname)
            
print('Building new file.')
fileout = nc.Dataset(fullname, 'w')
long_dim = fileout.createDimension('long', 360)
lat_dim = fileout.createDimension('lat', 280)

dim_tuple = ("long","lat")
pval_8007_5179 = fileout.createVariable("pval_8007_5179", 'f8', dim_tuple)
pval_9407_8093 = fileout.createVariable("pval_9407_8093", 'f8', dim_tuple)
    
pvals = np.zeros([360,280,2])
    
#P-VALUES OF TOTAL RAINFALL
for p1_b, p1_e, p2_b, p2_e, k in zip(p1_yr_b, p1_yr_e, p2_yr_b, p2_yr_e, np.arange(len(p1_yr_b))): 
    for i in range(360):
        
        start1 = time.time()
        print('lat: ', str(i))
        
        for j in range(280):
            if ma.getmask(P_asia[i,j,0]) == 0:
                
                #print('lon: ', str(j))
                pvals[i,j,k] = bs_means_diff(P_asia[i,j,p2_b-1:p2_e],\
                                             P_asia[i,j,p1_b-1:p1_e], niter)[1]
                
        end1 = time.time()
        print(end1-start1)
        
pval_8007_5179[:] = ma.array(pvals[:,:,0], mask = ma.getmask(P_asia[:,:,0]))
pval_9407_8093[:] = ma.array(pvals[:,:,1], mask = ma.getmask(P_asia[:,:,0]))

fileout.close()

#backup to external directory
if os.path.exists(backupname):
    os.remove(backupname)
copyfile(fullname, backupname) #backs file up in case of corruption

Building new file.
lat:  0
19.988147974014282
lat:  1
22.220031023025513
lat:  2
20.31821084022522
lat:  3
20.5697660446167
lat:  4
20.46646213531494
lat:  5
20.631201028823853
lat:  6
20.085623025894165
lat:  7
21.40938711166382
lat:  8
21.081300020217896
lat:  9
21.394248962402344
lat:  10
20.07054090499878
lat:  11
20.366620779037476
lat:  12
19.728087186813354
lat:  13
19.927491188049316
lat:  14
20.069051027297974
lat:  15
19.522921800613403
lat:  16
20.24071478843689
lat:  17
20.58786392211914
lat:  18
20.95716619491577
lat:  19
20.560920000076294
lat:  20
20.31446886062622
lat:  21
19.60966420173645
lat:  22
20.976773977279663
lat:  23
19.254042148590088
lat:  24
19.113651990890503
lat:  25
20.949375867843628
lat:  26
22.46813201904297
lat:  27
20.02254319190979
lat:  28
19.702373027801514
lat:  29
20.878720998764038
lat:  30
22.730136156082153
lat:  31
20.59200382232666
lat:  32
22.13016676902771
lat:  33
24.17644691467285
lat:  34
23.73519015312195
lat:  35
22.026401042938232


'/Users/Siwen/Desktop/Ferret/bin/asia_rain_diff_pval_2000iter.nc'